In [4]:
import pandas as pd

# Промежуточный МиниПроект в рамках обучения по курсу анализа данных от karpov.courses

Задание: Продакт-менеджер Василий попросил вас проанализировать завершенные уроки и ответить на следующие вопросы:

1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) (7 баллов).

2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*. (5 баллов)

3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). (5 баллов) 

4. Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3). (8 баллов)

5. Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов. (10 баллов) 

6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 0.1 до 0.5, monetary от 55 до 72 баллов. Описание подхода можно найти тут. (35 баллов)

*завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

Описание Файлов: 

assessments.csv — этот файл содержит информацию об оценках в тесте. Обычно каждый предмет в семестре включает ряд тестов с оценками, за которыми следует заключительный экзаменационный тест (экзамен).
code_module — идентификационный код предмета. code_presentation — семестр (Идентификационный код).
id_assessment — тест (Идентификационный номер ассессмента).
assessment_type — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).
date — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).
weight — вес теста в % в оценке за курс. Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%.

courses.csv — файл содержит список предметов по семестрам.
code_module — предмет (идентификационный код).
code_presentation — семестр (идентификационный код).
module_presentation_length — продолжительность семестра в днях.

studentAssessment.csv — этот файл содержит результаты тестов студентов. Если учащийся не отправляет работу на оценку, результат не записывается в таблицу.
id_assessment — тест (идентификационный номер).
id_student — идентификационный номер студента.
date_submitted — дата сдачи теста студентом, измеряемая как количество дней с начала семестра.
is_banked — факт перезачета теста с прошлого семестра (иногда курсы перезачитывают студентам, вернувшимся из академического отпуска).
score — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 неудачная/неуспешная сдача теста.

studentRegistration.csv — этот файл содержит информацию о времени, когда студент зарегистрировался для прохождения курса в семестре.
code_module — предмет (идентификационный код).
code_presentation — семестр (идентификационный код)
id_student — идентификационный номер студента.
date_registration — дата регистрации студента. Это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).
date_unregistration — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.

In [33]:
# Считываем все файлы
# Сори что считываю файл так. Просто я отстал, не успеваю разобраться с API
assessments = pd.read_csv('https://downloader.disk.yandex.ru/disk/05b574c8c2582fa1defdee4ad90a0b1fbc587154ddab900be97f5631941ecd48/63ce5303/I7TlrcXn7vP2NSW6_SzAKHtyBU5UQvPJYb9WaS9jyrX1CN8aOLdYoTLoJ5y31abgHM7JOy4RPGiMPia3fuppPQ%3D%3D?uid=0&filename=assessments.csv&disposition=attachment&hash=NsU8bgYQWMX58m9CL%2B6b8Jlb0Yp2tx3DNQvbSOAcusphkG2psyY%2BNP/GpIpMq3pSq/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=text%2Fplain&owner_uid=1676086898&fsize=8200&hid=07361bb7413da6d033c85305add628df&media_type=spreadsheet&tknv=v2')
courses = pd.read_csv('https://downloader.disk.yandex.ru/disk/a0cdd743ff07c04082d0580c5ecab18baaf2b42fb54ad0f984ef57bd0dc9fe20/63ce53c9/I7TlrcXn7vP2NSW6_SzAKD0eE9aMmQztCoU0FwX3IP-5_tDbRlk_Ejyr3J544g6J9Ztbosv84_sfrnjSGgxi_A%3D%3D?uid=0&filename=courses.csv&disposition=attachment&hash=kwzv96d7rpNypTIF4UrUhiTRSGymmYvsHXDZxiUqNx7155e9DWJ6/U9/7lJTU76Vq/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=text%2Fplain&owner_uid=1676086898&fsize=526&hid=5e60f4545117c54d9704641c012e6d8f&media_type=spreadsheet&tknv=v2')
studentAssessment = pd.read_csv('https://downloader.disk.yandex.ru/disk/244a85fab4bb50ac6970b1578d230e490bfb07610d411225c90b34a94952bd49/63ce5402/I7TlrcXn7vP2NSW6_SzAKJGGY1_jzNtybm4QOEPvowtv6BRTl8538-5c4_UXNoLYEQF253dO-bpK6mA4Mk9R1A%3D%3D?uid=0&filename=studentAssessment.csv&disposition=attachment&hash=QU%2BDEytZKz8bGczkF38UPLolHtYJ4WF7uZaQXEi5Ca0dmHlFj13peFhqLVI/qOG4q/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=text%2Fplain&owner_uid=1676086898&fsize=5690310&hid=410782fac70dfdb8c6be36efa364fe68&media_type=spreadsheet&tknv=v2')
studentRegistration = pd.read_csv('https://downloader.disk.yandex.ru/disk/4cc5bbc7242c660639fbb132a6f3606b0db085e22a5de91acc6ae8516c6daf86/63ce5430/I7TlrcXn7vP2NSW6_SzAKOV0_ccHM8DY9ZSnN7pb2CTgOkdRRMAr7jDOg_YNXVsWgEcr6OZwgOW57bQ6vpx0LA%3D%3D?uid=0&filename=studentRegistration.csv&disposition=attachment&hash=SwqwzHxJVUK5WojZvdDYa4OVEEI0YDbqkoyv%2BgJTbRxNjqwE8QVg01qAwNMg8dpNq/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=text%2Fplain&owner_uid=1676086898&fsize=1109984&hid=8afee2501fe1ddaf7446cd6f3cc0b6cf&media_type=spreadsheet&tknv=v2')

# 1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене)

In [6]:
# Не совсем понятно как соотносятся понятия курс, модуль, предмет.
# В задании вместо слова курс напишите пожалуйста "модуль" или "предмет", ато я потратил кучу времени
# думая что курс это учебный год)))

In [7]:
# К таблице 'studentAssessment' джойню 'assessments' чтобы отобрать только экзамены по 'assessment_type'

# не нашел как джойнить чтобы оставить только нужные колонки, а искать не успеваю, поэтому джойню все колонки

studentAssessment_merge =studentAssessment.merge(assessments, how='inner', on='id_assessment')

In [8]:
# отбираю только экзамены и с баллами >= 40. Группирую по id студента,
# агрегирую количество уникальных 'code_module'. Сортирую и вижу что максимальное значение = 2. Странно...
studentAssessment_merge \
    .query('assessment_type == "Exam"') \
    .query('score >= 40') \
    .groupby('id_student', as_index=False) \
    .agg({'code_module': pd.Series.nunique}) \
    .query('code_module < 2') \
    .sort_values('id_student')


# пробую сохранить ДФ без агрегации и группировки и проверяю подсчет уникальных сданных предметов
# еще двумя способами. Получаю точно такой же результат

# ПЕРВЫЙ СПОСОБ
# funcs = dict(code_module="nunique")
# dff.groupby(pd.Grouper(key="id_student")).agg(funcs).sort_values('code_module')

# ВТОРОЙ СПОСОБ
# dff.groupby('id_student').code_module.nunique().sort_values()

# Убеждаюсь что максимальное кол-во сданных экзаменов у студентов = 2

# ОТВЕТ на 1-й вопрос 3805 студентов

,id_student,code_module
0,23698,1
1,24213,1
2,27116,1
3,28046,1
4,28787,1
...,...,...
4092,2694886,1
4093,2694933,1
4094,2695608,1
4095,2697181,1


# 2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью

In [9]:
# С помощью лямбда функции добавляю колонку где если score >= 40 записываем 1 а если меньше то 0
studentAssessment_merge['success_end'] = studentAssessment_merge.score.apply(lambda x: 1 if x >= 40 else 0)

In [10]:
# оставляю только экзамены, группирую по id предмета, агрегирую количество тех кто вообще пробовал сдавать
# и количество тех кто сдал успешно, то есть сумму тех у кого бал выше 40. Сохраняю в отдельном ДФ, чтобы посчитать %
success_end_exam = studentAssessment_merge \
    .query('assessment_type == "Exam"') \
    .groupby('id_assessment', as_index=False) \
    .agg({'id_student': pd.Series.nunique, 'success_end': sum})

In [11]:
# добавляю колонку с соотношением успешных экзаменов
success_end_exam ['percent_success'] = (success_end_exam.success_end / success_end_exam.id_student).round(3)

In [12]:
# самая низкая завершаемость у 25340 = 0.837
# самая высокая завершаемость у 25361 = 0.926
success_end_exam.sort_values('percent_success')

,id_assessment,id_student,success_end,percent_success
2,25340,602,504,0.837
1,24299,1168,1019,0.872
5,25368,950,842,0.886
0,24290,747,664,0.889
3,25354,968,878,0.907
4,25361,524,485,0.926


# 3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом)

In [13]:
# Слишком всё просто, чую подвох
studentAssessment_merge \
    .query('assessment_type == "Exam"') \
    .query('score >= 40') \
    .groupby('code_module', as_index=False) \
    .agg({'date_submitted': 'mean'}) \
    .sort_values('date_submitted')

,code_module,date_submitted
1,DDD,237.976006
0,CCC,239.345217


# 4. Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3)

In [14]:
studentRegistration_top = studentRegistration \
    .groupby('code_module', as_index=False) \
    .agg({'id_student': pd.Series.nunique, 'date_unregistration': 'count'}) \
    .sort_values('id_student', ascending=False)

In [15]:
# Топ три курса по количеству регистраций: BBB, FFF, DDD
studentRegistration_top

,code_module,id_student,date_unregistration
1,BBB,7692,2377
5,FFF,7397,2380
3,DDD,5848,2235
2,CCC,4251,1947
4,EEE,2859,718
6,GGG,2525,289
0,AAA,712,126


In [16]:
# Добавляем колонку: количество отмен деленное на количество всего регистраций на предмет
# Топ-3 по оттоку: CCC, DDD, FFF
studentRegistration_top ['outflow'] = studentRegistration_top.date_unregistration / studentRegistration_top.id_student
studentRegistration_top.sort_values('outflow', ascending=False)

,code_module,id_student,date_unregistration,outflow
2,CCC,4251,1947,0.458010
3,DDD,5848,2235,0.382182
5,FFF,7397,2380,0.321752
1,BBB,7692,2377,0.309022
4,EEE,2859,718,0.251137
0,AAA,712,126,0.176966
6,GGG,2525,289,0.114455


# 5. Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов

In [17]:
studentAssessment_merge.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,success_end
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0,1
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0,1
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0,1
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0,1
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0,1


In [18]:
# Тут я не совсем понимаю для чего нужна функция, возможно не понял смысл задания
# Поэтому решал так: оставил только экзамены, сгруппировал по семестрам и посчитал:
# 1) средний срок сдачи 2) сколько всего курсов в семестре
# и сколько курсов завершенных с баллом выше 40. На их основе посчитаю завершаемость
# В колонке success_end стоит 1 если курс сдал на 40+ баллов. Поэтому для подсчета успешных считаем 'sum'

cohort = studentAssessment_merge \
    .query ('assessment_type == "Exam"') \
    .groupby ('code_presentation', as_index=False) \
    .agg ({'date_submitted': 'mean', 'score': 'count', 'success_end': 'sum'})
cohort

,code_presentation,date_submitted,score,success_end
0,2013B,230.179402,602,504
1,2013J,239.505165,968,878
2,2014B,232.988985,1271,1149
3,2014J,243.674221,2118,1861


In [19]:
# Семестр с самой низкой завершаемостью курсов - 2013B
# Семестр с самым долгим средним сроком сдачи курсов - 2014J

cohort ['finalizability'] = cohort.success_end / cohort.score
cohort = cohort.drop(['score', 'success_end'], 1)
cohort.rename(columns={'date_submitted': 'avg_acourse_completion_time'}, inplace=True)
cohort

,code_presentation,avg_acourse_completion_time,finalizability
0,2013B,230.179402,0.837209
1,2013J,239.505165,0.907025
2,2014B,232.988985,0.904013
3,2014J,243.674221,0.878659


# 6. RFM  анализ

In [20]:
studentAssessment_merge.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,success_end
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0,1
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0,1
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0,1
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0,1
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0,1


In [21]:
# Незнаю правильно ли, но несданные экзамены (меньше 40 баллов) тоже учитываю в этом задании

# Пока сагрегирую нужные показатели и положу в отдельный ДФ

# date_submitted: mean - среднее время сдачи одного экзамена
# success_end: sum - кол-во успешных экзаменов. Использую sum, т.к. у неуспешных значение 0, у успешных 1
# assessment_type: count - кол-во всех попыток сдать экзамен. Т.к. ДФ отфильтрован по 'Exam', просто считаем их кол-во
# score: mean - среднее количество баллов, получаемое за экзамен

rfm = studentAssessment_merge \
    .query('assessment_type == "Exam"') \
    .groupby('id_student', as_index=False) \
    .agg({'date_submitted': 'mean', 'success_end': 'sum', 'assessment_type': 'count', 'score': 'mean'})
rfm.head()

,id_student,date_submitted,success_end,assessment_type,score
0,23698,243.0,1,1,80.0
1,24213,236.0,1,1,58.0
2,27116,243.0,1,1,96.0
3,28046,237.0,1,1,40.0
4,28787,243.0,1,1,44.0


In [22]:
rfm.insert(2, "finalizability", rfm.success_end / rfm.assessment_type) # считаю завершаемость, колонку ставлю третьей

rfm = rfm.drop(['success_end', 'assessment_type'], 1) # удаляю колонки, т.к. завершаемость уже посчитана

rfm.rename(columns={'date_submitted': 'avg_time_exam', 'score': 'avg_score_exam'}, inplace=True) # переименовываю

# avg_time_exam - R - среднее время сдачи одного экзамена
# finalizability - F - завершаемость курсов
# avg_score_exam - M - среднее количество баллов, получаемое за экзамен

In [23]:
# теперь все данные готовы
rfm

,id_student,avg_time_exam,finalizability,avg_score_exam
0,23698,243.0,1.0,80.0
1,24213,236.0,1.0,58.0
2,27116,243.0,1.0,96.0
3,28046,237.0,1.0,40.0
4,28787,243.0,1.0,44.0
...,...,...,...,...
4628,2694886,236.0,1.0,69.0
4629,2694933,230.0,1.0,73.0
4630,2695608,237.0,1.0,73.0
4631,2697181,230.0,1.0,80.0


In [24]:
# Теперь нужно создать кластеры. По каждому показателю пусть будет три интервала.

# Для 'avg_time_exam' и 'avg_score_exam' логично использовать персентили. 
# По 'avg_time_exam' - 1-я группа до 0.33 персентиля включительно (<=236), 2-я группа до 0.66 (<=243), остальное 3-я группа
# По 'avg_score_exam' наоборот - до 0.33 персентиля (<=56) - 3 группа, до 0.66 (<=76) - 2 группа, остальное 1 группа

# По 'finalizability' сейчас есть всего три значения 1, 0 и 0.5. Но это из-за того что максимальное число экзаменов
# пока что только 2. Т.к. возможны и значения 0.1 или 0.7 или другие, логично тоже просто разделить на 3 группы по 33%
# Чем ближе к единице тем лучше завершаемость: до 0.33 - 3 группа, до 0.66 - 2 группа, остальные 1 группа

In [25]:
# Предлагаю лямбда функциями добавить соответствующие колонки
rfm ['R'] = rfm.avg_time_exam.apply(lambda x:
                                    1 if x <= rfm.avg_time_exam.quantile(0.33) else (
                                        2 if x <= rfm.avg_time_exam.quantile(0.66) 
                                        else 3))


rfm ['F'] = rfm.avg_score_exam.apply(lambda x: 
                                     3 if x <= rfm.avg_score_exam.quantile(0.33) else (
                                         2 if x <= rfm.avg_score_exam.quantile(0.66) 
                                         else 1))


rfm ['M'] = rfm.finalizability.apply(lambda x: 
                                     3 if x <= 0.33 else (
                                         2 if x <= 0.66 
                                         else 1))

In [26]:
# Вот что получилось
rfm.head()

,id_student,avg_time_exam,finalizability,avg_score_exam,R,F,M
0,23698,243.0,1.0,80.0,2,1,1
1,24213,236.0,1.0,58.0,1,2,1
2,27116,243.0,1.0,96.0,2,1,1
3,28046,237.0,1.0,40.0,2,3,1
4,28787,243.0,1.0,44.0,2,3,1


In [27]:
# Попробую вывести разные кластеры. Самые успешные: быстрее всех сдают экзамены, с хорошей завершаемостью и высокими баллами
rfm.query('R == 1 and F == 1 and M ==1')

,id_student,avg_time_exam,finalizability,avg_score_exam,R,F,M
8,29764,230.0,1.0,94.0,1,1,1
25,46605,230.0,1.0,98.0,1,1,1
28,49119,234.0,1.0,100.0,1,1,1
29,50476,234.0,1.0,87.0,1,1,1
53,67781,234.0,1.0,84.0,1,1,1
...,...,...,...,...,...,...,...
4614,2685827,234.0,1.0,84.0,1,1,1
4624,2692327,234.0,1.0,88.0,1,1,1
4625,2692577,234.0,1.0,87.0,1,1,1
4627,2693243,230.0,1.0,82.0,1,1,1


In [28]:
# Отстающие: сдают экзамены дольше всех, завершаемость низкая, баллы низкие
rfm.query('R == 3 and F == 3 and M ==3')

,id_student,avg_time_exam,finalizability,avg_score_exam,R,F,M
63,72609,245.0,0.0,24.0,3,3,3
75,81351,248.0,0.0,0.0,3,3,3
105,109577,245.0,0.0,26.0,3,3,3
118,120236,244.0,0.0,26.0,3,3,3
137,131152,244.0,0.0,0.0,3,3,3
...,...,...,...,...,...,...,...
4331,2232340,249.0,0.0,36.0,3,3,3
4406,2405934,244.0,0.0,28.0,3,3,3
4440,2458925,245.0,0.0,28.0,3,3,3
4470,2499779,244.0,0.0,36.0,3,3,3


In [29]:
# Середнячков сегмента 222 нет, есть только 221
rfm.query('R == 2 and F == 2 and M ==1')

,id_student,avg_time_exam,finalizability,avg_score_exam,R,F,M
5,29411,237.0,1.0,62.0,2,2,1
15,33930,237.0,1.0,67.0,2,2,1
19,40419,237.0,1.0,69.0,2,2,1
39,57227,242.0,1.0,67.0,2,2,1
43,59370,242.0,1.0,60.0,2,2,1
...,...,...,...,...,...,...,...
4609,2678380,242.0,1.0,64.0,2,2,1
4612,2684043,243.0,1.0,69.0,2,2,1
4620,2689863,242.0,1.0,67.0,2,2,1
4623,2692186,237.0,1.0,58.0,2,2,1
